#### NO CORRER CON RUN ALL

In [1]:
import sys
from pathlib import Path
import os
from pathlib import Path

print(os.getcwd())
script_dir = Path().resolve().parent
os.chdir(script_dir)
print(os.getcwd())


d:\Facultad\AA\aprendizaje_automatico\TP2
D:\Facultad\AA\aprendizaje_automatico


In [2]:
from pathlib import Path

RANDOM_SEED = 42
dataset_root = "TP2/dataset/guardar_datos"
dataset_path = Path(dataset_root)
train_parquet_path = dataset_path / "train_dataset.parquet"
valid_parquet_path = dataset_path / "valid_dataset.parquet"

## CORRER ESTA CELDA SOLO UNA VEZ
# 50 GB WARNING 

In [ ]:
import os.path


import torch
import torch.nn as nn

import pandas as pd
from TP2.tokenizer import dataset_to_tokens

import os
from multiprocessing import Pool, cpu_count
from torch.utils.data import DataLoader

def process_single_batch(args):
    """Worker function that processes one batch."""
    batch_data, batch_idx, split_name, dataset_path = args
    tokenized = dataset_to_tokens(batch_data)
    output_path = os.path.join(dataset_path, f"{split_name}_tokens_batch_{batch_idx}.parquet")
    tokenized.to_parquet(output_path)
    return batch_idx


def batch_tokenizer():
    TOKENIZER_BATCH_SIZE = 8192

    train_df = pd.read_parquet(train_parquet_path)
    train_labels = train_df['X']
    print(f"Train instances: {len(train_labels)}")
    train_tasks = [
        (train_labels[i:i + TOKENIZER_BATCH_SIZE], i // TOKENIZER_BATCH_SIZE, 'train', dataset_path)
        for i in range(0, len(train_labels), TOKENIZER_BATCH_SIZE)
    ]

    valid_df = pd.read_parquet(valid_parquet_path)
    valid_labels = valid_df['X']
    print(f"Validation instances: {len(valid_labels)}")
    valid_tasks = [
        (valid_labels[i:i + TOKENIZER_BATCH_SIZE], i // TOKENIZER_BATCH_SIZE, 'valid', dataset_path)
        for i in range(0, len(valid_labels), TOKENIZER_BATCH_SIZE)
    ]

    all_tasks = train_tasks + valid_tasks
    with Pool(processes=6) as pool:
        results = pool.map(process_single_batch, all_tasks)

    print(f"Completed {len(results)} batches using {cpu_count()} cores")

# yo no lo descargo porque ya lo tengo
batch_tokenizer()
exit()


Train instances: 180625
Validation instances: 31875


In [3]:
INSTANCE_COUNT = 20000
BATCH_SIZE = 8192

In [12]:
from TP2.models.RNN_LSTM import RNN_LSTM
from TP2.dataloader import TextDataset
from TP2.training import training_loop
import torch.nn as nn
from torch.utils.data import DataLoader


train_dataset = TextDataset(
    base_path= dataset_root,
    parquet_file_subpath="train_tokens_batch_",
    batch_size=BATCH_SIZE,
    instance_count=INSTANCE_COUNT#252875 #TODO arreglar que no coincide con el print!!!!!!!!!!!!!!!!!!!!!!!!!!!11 (creo que ya está, ver la celda cursed del notebook separar datos)
    #instance_count=291346  # ajustar segun la cantidad de instancias en el dataset
)
train_dataloader = DataLoader(train_dataset, batch_size=32, collate_fn=train_dataset.collate_fn) # NO HACER SHUFFLE!


valid_dataset = TextDataset(
    base_path= dataset_root,
    parquet_file_subpath="valid_tokens_batch_",
    batch_size=BATCH_SIZE,
    instance_count=INSTANCE_COUNT#44625 #TODO arreglar que no coincide con el print!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    #instance_count=51415  # ajustar segun la cantidad de instancias en el dataset
)
valid_dataloader = DataLoader(valid_dataset, batch_size=256, collate_fn=valid_dataset.collate_fn) # NO HACER SHUFFLE! TODO explicar por qué no shuffle

model = RNN_LSTM(embedding_dim=768, hidden_dim=256, num_layers=2, bidirectional=True)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

# Si hay multiples GPUs, usamos DataParallel para envolver el modelo y poder utilizarlas
if torch.cuda.device_count() > 1:
    print("Usando", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model)

model.to(device)


history = training_loop(model, device, train_dataloader, valid_dataloader, criterion, optimizer, num_epochs=20)


#torch.save(model.state_dict(), "rnn_simple_model.pth")


Epoch 1/20


100%|██████████| 625/625 [02:00<00:00,  5.17it/s]


Train Loss: 0.4523 | F1 Punct Ini: 0.5931 | F1 Punct Fin: 0.5141 | F1 Cap: 0.5653
Val   Loss: 0.3704 | F1 Punct Ini: 0.6596 | F1 Punct Fin: 0.5372 | F1 Cap: 0.6244

Epoch 2/20


 13%|█▎        | 82/625 [00:22<02:31,  3.58it/s] 


KeyboardInterrupt: 